# Content-Based Recommendation Systems

Referenced code from: Banik, Rounak. 2018. Hands-On Recommendation Systems with Python: Start building powerful and personalized, recommendation engines with Python. Packt Publishing.

In [150]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import wordnet
from ast import literal_eval
import progressbar
from time import sleep
import numpy as np
df = pd.read_csv('/Users/connorranson/Downloads/movies_metadata_subset.csv')
df = df.drop(columns=['idd'])
links = pd.read_csv('/Users/connorranson/Downloads/links.csv')

In [70]:
df = pd.merge(df, links, left_on='id', right_on='tmdbId')
print (df.shape)
df.head()


(9555, 11)


,id,title,genres,release_date,runtime,vote_average,vote_count,year,movieId,imdbId,tmdbId
0,862,Toy Story,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",1995-10-30,81.0,7.7,5415.0,1995,1,114709,862.0
1,8844,Jumanji,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",1995-12-15,104.0,6.9,2413.0,1995,2,113497,8844.0
2,15602,Grumpier Old Men,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",1995-12-22,101.0,6.5,92.0,1995,3,113228,15602.0
3,31357,Waiting to Exhale,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",1995-12-22,127.0,6.1,34.0,1995,4,114885,31357.0
4,11862,Father of the Bride Part II,"[{'id': 35, 'name': 'Comedy'}]",1995-02-10,106.0,5.7,173.0,1995,5,113041,11862.0


In [71]:
df1 = pd.read_csv('/Users/connorranson/Downloads/movies_metadata.csv')
df1 = df1[['id','title','genres', 'release_date', 'runtime', 'vote_average', 'vote_count','overview']]
df1.head()

/Users/connorranson/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,id,title,genres,release_date,runtime,vote_average,vote_count,overview
0,862,Toy Story,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",1995-10-30,81.0,7.7,5415.0,"Led by Woody, Andy's toys live happily in his ..."
1,8844,Jumanji,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",1995-12-15,104.0,6.9,2413.0,When siblings Judy and Peter discover an encha...
2,15602,Grumpier Old Men,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",1995-12-22,101.0,6.5,92.0,A family wedding reignites the ancient feud be...
3,31357,Waiting to Exhale,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",1995-12-22,127.0,6.1,34.0,"Cheated on, mistreated and stepped on, the wom..."
4,11862,Father of the Bride Part II,"[{'id': 35, 'name': 'Comedy'}]",1995-02-10,106.0,5.7,173.0,Just when George Banks has recovered from his ...


In [72]:
df['overview'], df['id'] = df1['overview'], df1['id']

In [73]:
df.head()


,id,title,genres,release_date,runtime,vote_average,vote_count,year,movieId,imdbId,tmdbId,overview
0,862,Toy Story,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",1995-10-30,81.0,7.7,5415.0,1995,1,114709,862.0,"Led by Woody, Andy's toys live happily in his ..."
1,8844,Jumanji,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",1995-12-15,104.0,6.9,2413.0,1995,2,113497,8844.0,When siblings Judy and Peter discover an encha...
2,15602,Grumpier Old Men,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",1995-12-22,101.0,6.5,92.0,1995,3,113228,15602.0,A family wedding reignites the ancient feud be...
3,31357,Waiting to Exhale,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",1995-12-22,127.0,6.1,34.0,1995,4,114885,31357.0,"Cheated on, mistreated and stepped on, the wom..."
4,11862,Father of the Bride Part II,"[{'id': 35, 'name': 'Comedy'}]",1995-02-10,106.0,5.7,173.0,1995,5,113041,11862.0,Just when George Banks has recovered from his ...


In [74]:
#Import the literal_eval function from ast
from ast import literal_eval

#Define a stringified list and output its type
a = "[1,2,3]"
print(type(a))

#Apply literal_eval and output type
b = literal_eval(a)
print(type(b))

<class 'str'>
<class 'list'>


In [75]:
df['genres'] = df['genres'].fillna('[]')

#Apply literal_eval to convert stringified empty lists to the list object
df['genres'] = df['genres'].apply(literal_eval)

#Convert list of dictionaries to a list of strings
df['genres'] = df['genres'].apply(lambda x: [i['name'].lower() for i in x] if isinstance(x, list) else [])

In [149]:
df.to_csv('df_gen.csv', index=False)

In [77]:
df = df.drop(columns = ['vote_average', 'vote_count'])
df.shape

(9555, 10)

In [78]:
#Import TfIdfVectorizer from the scikit-learn library
from sklearn.feature_extraction.text import TfidfVectorizer

#Define a TF-IDF Vectorizer Object. Remove all english stopwords
tfidf = TfidfVectorizer(stop_words='english')

#Replace NaN with an empty string
df['overview'] = df['overview'].fillna('')

#Construct the required TF-IDF matrix by applying the fit_transform method on the overview feature
tfidf_matrix = tfidf.fit_transform(df['overview'])

#Output the shape of tfidf_matrix
tfidf_matrix.shape

(9555, 31558)

In [79]:
#Create a new feature by exploding genres
s = df.apply(lambda x: pd.Series(x['genres']),axis=1).stack().reset_index(level=1, drop=True)

#Name the new feature as 'genre'
s.name = 'genre'

#Create a new dataframe gen_df which by dropping the old 'genres' feature and adding the new 'genre'.
gen_df = df.drop('genres', axis=1).join(s)

#Print the head of the new gen_df
gen_df.head()

/Users/connorranson/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  


,id,title,release_date,runtime,year,movieId,imdbId,tmdbId,overview,genre
0,862,Toy Story,1995-10-30,81.0,1995,1,114709,862.0,"Led by Woody, Andy's toys live happily in his ...",animation
0,862,Toy Story,1995-10-30,81.0,1995,1,114709,862.0,"Led by Woody, Andy's toys live happily in his ...",comedy
0,862,Toy Story,1995-10-30,81.0,1995,1,114709,862.0,"Led by Woody, Andy's toys live happily in his ...",family
1,8844,Jumanji,1995-12-15,104.0,1995,2,113497,8844.0,When siblings Judy and Peter discover an encha...,adventure
1,8844,Jumanji,1995-12-15,104.0,1995,2,113497,8844.0,When siblings Judy and Peter discover an encha...,fantasy


In [80]:
# Import linear_kernel to compute the dot product
from sklearn.metrics.pairwise import linear_kernel

# Compute the cosine similarity matrix
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

In [81]:
#Construct a reverse mapping of indices and movie titles, and drop duplicate titles, if any
indices = pd.Series(df.index, index=df['title']).drop_duplicates()

In [82]:
# Function that takes in movie title as input and gives recommendations 
def content_recommender(title, cosine_sim=cosine_sim, df=df, indices=indices):
    # Obtain the index of the movie that matches the title
    idx = indices[title]

    # Get the pairwsie similarity scores of all movies with that movie
    # And convert it into a list of tuples as described above
    sim_scores = list(enumerate(cosine_sim[idx]))

    # Sort the movies based on the cosine similarity scores
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Get the scores of the 10 most similar movies. Ignore the first movie.
    sim_scores = sim_scores[1:11]

    # Get the movie indices
    movie_indices = [i[0] for i in sim_scores]

    # Return the top 10 most similar movies
    return df['title'].iloc[movie_indices]

In [83]:
content_recommender('Jumanji')

6166                          Stay Alive
8801               Shaun the Sheep Movie
9503    The Godfather Trilogy: 1972-1990
8079                       The Liability
6055                       Walk the Line
2486                         Wonder Boys
9107                                Eros
1506                  Blackbeard's Ghost
7749                  Bullet to the Head
7906     Dragon Ball Z: Return of Cooler
Name: title, dtype: object

## MetaData Recommender

In [84]:
tags = pd.read_csv('/Users/connorranson/Downloads/tags.csv')

In [85]:
tags.head()

,userId,movieId,tag,timestamp
0,2,60756,funny,1445714994
1,2,60756,Highly quotable,1445714996
2,2,60756,will ferrell,1445714992
3,2,89774,Boxing story,1445715207
4,2,89774,MMA,1445715200


In [86]:
movies = pd.read_csv('/Users/connorranson/Downloads/movies.csv')

In [87]:
movie_tag = pd.merge(tags, movies, on = 'movieId')
movie_tag.head()

,userId,movieId,tag,timestamp,title,genres
0,2,60756,funny,1445714994,Step Brothers (2008),Comedy
1,2,60756,Highly quotable,1445714996,Step Brothers (2008),Comedy
2,2,60756,will ferrell,1445714992,Step Brothers (2008),Comedy
3,62,60756,comedy,1528934384,Step Brothers (2008),Comedy
4,62,60756,funny,1528934381,Step Brothers (2008),Comedy


In [88]:
links = pd.read_csv('/Users/connorranson/Downloads/links.csv')

In [89]:
tag_links = pd.merge(tags, links, on = 'movieId')
tag_links.head()

,userId,movieId,tag,timestamp,imdbId,tmdbId
0,2,60756,funny,1445714994,838283,12133.0
1,2,60756,Highly quotable,1445714996,838283,12133.0
2,2,60756,will ferrell,1445714992,838283,12133.0
3,62,60756,comedy,1528934384,838283,12133.0
4,62,60756,funny,1528934381,838283,12133.0


In [90]:
def clean_ids(x):
    try:
        return int(x)
    except:
        return np.nan

In [91]:
#Clean the ids of df
tag_links['tmdbId'] = tag_links['tmdbId'].apply(clean_ids)

#Filter all rows that have a null ID
tag_links = tag_links[tag_links['tmdbId'].notnull()]

In [98]:
#Clean the ids of df
df['id'] = df['id'].apply(clean_ids)

#Filter all rows that have a null ID
df = df[df['id'].notnull()]

In [99]:
tag_links.head()

,userId,movieId,tag,timestamp,imdbId,tmdbId
0,2,60756,funny,1445714994,838283,12133
1,2,60756,Highly quotable,1445714996,838283,12133
2,2,60756,will ferrell,1445714992,838283,12133
3,62,60756,comedy,1528934384,838283,12133
4,62,60756,funny,1528934381,838283,12133


In [100]:
df_tag = pd.merge(df, tag_links, left_on = "id", right_on = "tmdbId")
df_tag = df_tag.drop(columns=['movieId_y', 'timestamp', 'imdbId_y','tmdbId_y'])
df_tag = df_tag.rename(columns={"movieId_x": "movieId", "imdbId_x": "imdbId", "tmdbId_x": "tmdbId"})
df_tag.head()

,id,title,genres,release_date,runtime,year,movieId,imdbId,tmdbId,overview,userId,tag
0,862,Toy Story,"[animation, comedy, family]",1995-10-30,81.0,1995,1,114709,862.0,"Led by Woody, Andy's toys live happily in his ...",336,pixar
1,862,Toy Story,"[animation, comedy, family]",1995-10-30,81.0,1995,1,114709,862.0,"Led by Woody, Andy's toys live happily in his ...",474,pixar
2,862,Toy Story,"[animation, comedy, family]",1995-10-30,81.0,1995,1,114709,862.0,"Led by Woody, Andy's toys live happily in his ...",567,fun
3,8844,Jumanji,"[adventure, fantasy, family]",1995-12-15,104.0,1995,2,113497,8844.0,When siblings Judy and Peter discover an encha...,62,fantasy
4,8844,Jumanji,"[adventure, fantasy, family]",1995-12-15,104.0,1995,2,113497,8844.0,When siblings Judy and Peter discover an encha...,62,magic board game


In [141]:
df_user_group = df_tag.groupby('userId').agg({'tag':lambda x: list(x)})
df_user_group.head()

,tag
userId,
18,"[Al Pacino, gangster, mafia, Al Pacino, Mafia,..."
21,"[romantic comedy, wedding]"
62,"[fantasy, magic board game, Robin Williams, be..."
63,"[classic, space action]"
76,"[action, sci-fi]"


In [101]:
df_group = df_tag.groupby('id').agg({'tag':lambda x: list(x)})

In [102]:
df_new = pd.merge(df, df_group, on = 'id')
df_new.head()

,id,title,genres,release_date,runtime,year,movieId,imdbId,tmdbId,overview,tag
0,862,Toy Story,"[animation, comedy, family]",1995-10-30,81.0,1995,1,114709,862.0,"Led by Woody, Andy's toys live happily in his ...","[pixar, pixar, fun]"
1,8844,Jumanji,"[adventure, fantasy, family]",1995-12-15,104.0,1995,2,113497,8844.0,When siblings Judy and Peter discover an encha...,"[fantasy, magic board game, Robin Williams, game]"
2,15602,Grumpier Old Men,"[romance, comedy]",1995-12-22,101.0,1995,3,113228,15602.0,A family wedding reignites the ancient feud be...,"[moldy, old]"
3,11862,Father of the Bride Part II,[comedy],1995-02-10,106.0,1995,5,113041,11862.0,Just when George Banks has recovered from his ...,"[pregnancy, remake]"
4,11860,Sabrina,"[comedy, romance]",1995-12-15,127.0,1995,7,114319,11860.0,An ugly duckling having undergone a remarkable...,[remake]


In [103]:
# Function to convert all non-integer IDs to NaN
def clean_ids(x):
    try:
        return int(x)
    except:
        return np.nan

In [104]:
#Clean the ids of df
df_new['id'] = df_new['id'].apply(clean_ids)

#Filter all rows that have a null ID
df_new = df_new[df_new['id'].notnull()]

In [105]:
# Function to sanitize data to prevent ambiguity. It removes spaces and converts to lowercase
def sanitize(x):
    if isinstance(x, list):
        #Strip spaces and convert to lowercase
        return [str.lower(i.replace(" ", "")) for i in x]
    else:
        #Check if director exists. If not, return empty string
        if isinstance(x, str):
            return str.lower(x.replace(" ", ""))
        else:
            return ''

In [106]:
#Apply the generate_list function to keywords, and genres
for feature in ['genres', 'tag']:
    df_new[feature] = df_new[feature].apply(sanitize)

In [130]:
#Function that creates a soup out of the desired metadata
def create_soup(x):
    return ' '.join(x['tag']) + ' '.join(x['genres'])

In [131]:
# Create the new soup feature
df_new['soup'] = df_new.apply(create_soup, axis=1)

In [132]:
#Display the soup of the first movie
df_new.iloc[0]['soup']

'pixar pixar funanimation comedy family'

In [133]:
# Import CountVectorizer
from sklearn.feature_extraction.text import CountVectorizer

#Define a new CountVectorizer object and create vectors for the soup
count = CountVectorizer(stop_words='english')
count_matrix = count.fit_transform(df_new['soup'])

In [134]:
#Import cosine_similarity function
from sklearn.metrics.pairwise import cosine_similarity

#Compute the cosine similarity score (equivalent to dot product for tf-idf vectors)
cosine_sim2 = cosine_similarity(count_matrix, count_matrix)

In [135]:
# Reset index of your df and construct reverse mapping again
df_new = df_new.reset_index()
indices2 = pd.Series(df_new.index, index=df_new['title'])

In [136]:
content_recommender('Jumanji', cosine_sim2, df_new, indices2)


55                                      A Little Princess
70                               The Secret of Roan Inish
587                              Harry and the Hendersons
1178    The Chronicles of Narnia: The Voyage of the Da...
310                                         A Simple Wish
461                                 The Boy Who Could Fly
497                                    My Science Project
654                The Adventures of Ichabod and Mr. Toad
1050            Harry Potter and the Order of the Phoenix
1084             The Chronicles of Narnia: Prince Caspian
Name: title, dtype: object